# CorrelAid for Klima*Kollektiv Karte der Transportleitung, Rohdungstrassen und Zerrstörungspunkte entlang der Trasse
Sep 27, 2024

by Tim Appelhans

In [60]:
# !pip install geopandas
# !pip install folium
# !pip install requests

In [61]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Polygon, LineString
import json

In [62]:
# ---- Visualization ----

# Create a base map
m = folium.Map(location = [51.0, 6.5], zoom_start = 9,  min_zoom = 8)

Add `Esri.WorldImagery` tile layer (see [Leaflet Provider Demo](https://leaflet-extras.github.io/leaflet-providers/preview/) for a quick preview of various basemaps).

In [63]:
# add Esri World Imagery tile layer
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri.WorldImagery',
    overlay=False,
    control=True,
    show=False # hide layer when opening the map
).add_to(m)

In [64]:
# load GeoPackage file 'transportleitungen_geom' into geodataframe gdf
gdf = gpd.read_file("transportleitung_geom.gpkg")

# print to inspect the data
gdf

,id,Name,geometry
0,0,transportleitung_west,"MULTILINESTRING ((350531.177 5662445.558, 3501..."
1,1,transportleitung_sued,"MULTILINESTRING ((333215.541 5658810.732, 3341..."


In [65]:
gdf = gdf.to_crs("EPSG:4326") # change CRS to the same as in 'consumer'
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [66]:

# Add the GeoDataFrame to the map
folium.GeoJson(
    gdf, 
    name="Water pipelines", # again, determine the name for the layer toggle
    style_function = lambda feature: {
        'color': 'orange' if feature['properties']['Name'] == 'transportleitung_sued' else 'green',
        'weight': 5
    },
    popup = folium.GeoJsonPopup(fields=['Name'])
).add_to(m)

# Add layer control to toggle layers
folium.LayerControl().add_to(m)

In [67]:
m

In [67]:
# save the map as hmtl
m.save('pipe_map.html')

---